In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import cv2
import imageio
from scipy.misc import imread

noisy_img_arr = cv2.imread('./images/binary_carl_noise.jpg')
img_arr = cv2.imread('./images/binary_carl.jpg')
cv2.imshow('test noise', noisy_img_arr)
cv2.imshow('test image', img_arr)

constant = [0,1,.02]
hidden_image = np.copy(noisy_img_arr)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
def percent_pixel_flipped(noisy_arr,denoised_arr):
    num_flipped = 0
    total_px = noisy_arr.shape[0]*noisy_arr.shape[1]
    for i in range(noisy_arr.shape[0]):
        for j in range(noisy_arr.shape[1]):
            if noisy_arr[i,j]!= denoised_arr[i,j]:
                num_flipped+=1
   
    percent_change = num_flipped*100./total_px
    return percent_change

def ProbY(y_array,x_array,X1, X2,i, j, constant):
    n = constant[2]
    energy = 0
    energy = -n * x_array[X1][X2] * y_array[i][j]
    prob = np.exp(-energy)
    #print("probY", prob)
    return prob

PX =[]
PXM = []
normX = []
normXM1 = []

def check_limit(value, limit):
    if value<0:
        value=limit-1
    if value==limit:
        value=0
    return value

def ProbX1(y_array,x_array,i, j, constant):
    energy = 0
    b = constant[1]
    x_neighbour = [-1,1]
    y_neighbour = [-1,1]
    energy = 0
    for k in x_neighbour:
        for l in y_neighbour:
            x_n = check_limit(i +k, x_array.shape[0])
            y_n = check_limit(j +l, x_array.shape[1])
            x_array[i][j] = 1
            energy += -b*x_array[i][j]*x_array[x_n][y_n]
    prob = np.exp(-energy)
    PX.append(prob)
    #print ("ProbX", prob)
    return prob

def ProbXM1(y_array,x_array,i, j, constant):
    energy = 0
    b = constant[1]
    x_neighbour = [-1,1]
    y_neighbour = [-1,1]
    energy = 0
    for k in x_neighbour:
        for l in y_neighbour:
            x_n = check_limit(i +k, x_array.shape[0])
            y_n = check_limit(j +l, x_array.shape[1])
            x_array[i][j] = -1
            energy += -b*x_array[i][j]*x_array[x_n][y_n]
    prob = np.exp(-energy)
    PXM.append(prob)
    #print ("ProbX", prob)
    return prob

def mainprob1(ProbX1, ProbXM1, ProbY, i, j):
    P1 = ProbX1(noisy_img_arr , hidden_image, i, j, constant)*ProbY(noisy_img_arr, hidden_image, 1, 1, i, j, constant)
    P2 = ProbXM1(noisy_img_arr , hidden_image, i, j, constant)*ProbY(noisy_img_arr, hidden_image, -1, -1, i, j, constant)
    #print("P1", P1)
    #print("P2", P2)
    P = P1/(P1+P2)
    return P

def mainprobM1(ProbX1, ProbXM1, ProbY, i, j):
    P1 = ProbX1(noisy_img_arr , hidden_image, i, j, constant)*ProbY(noisy_img_arr, hidden_image, 1, 1, i, j, constant)
    P2 = ProbXM1(noisy_img_arr , hidden_image, i, j, constant)*ProbY(noisy_img_arr, hidden_image, -1, -1, i, j, constant)
    #print("P1", P1)
    #print("P2", P2)
    P = P2/(P1+P2)
    return P

new_hidden_arr = np.copy(hidden_image)
# print ("% Pixels flipped:", percent_pixel_flipped(hidden_image, img_arr))

for T in range(5):
    for i in range(hidden_image.shape[0]):
        for j in range(hidden_image.shape[1]):
            #print("initial", hidden_image[0:10, 0:10])
            u = np.random.uniform(0,1)
            p = mainprob1(ProbX1, ProbXM1, ProbY, i, j)
            #print("j iteration", j)
            #print("u", u)
            #print("p", p)
            if p > u:
                hidden_image[i][j] = 1
            else:
                hidden_image[i][j] = -1
            #print("final", hidden_image[0:10, 0:10])
        #print("i iteration", i)
                       
#     print ("% Pixels flipped:", percent_pixel_flipped(hidden_image, img_arr))
    
cv2.imshow('hidden image', hidden_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\Farrel\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in double_scalars
